In [2]:
from pyspark import SparkConf, SparkContext
from collections import OrderedDict
from functools import partial

In [3]:
partitions = 24
parcsv = sc.textFile("/home/milroy/pyspark/2014-04-24.csv", partitions)
parcsv.take(5)

[u'oss07;lustre.scratch.ost.obdfilter.OST0017.cache_access;0.00000;float;pages/s;1398382546',
 u'oss07;lustre.scratch.ost.obdfilter.OST0015.disconnect;0.00000;float;requests/s;1398382546',
 u'oss07;cpu_intr;0.0;float;%;1398382546',
 u'oss07;lustre.scratch.ost.obdfilter.hosttotal.cache_access;0.00000;float;pages/s;1398382546',
 u'oss07;lustre.scratch.ost.obdfilter.OST0025.connect;0.00000;float;requests/s;1398382546']

In [4]:
filtered = parcsv.filter(lambda line: len(line.split(';')) == 6)

In [5]:
def cast(line):
    try:
      val = float(str(line.split(';')[2]))
    except:
      val = 0.0
    return (int(line.split(';')[5]), str(line.split(';')[0]), 
            str(line.split(';')[1]), val)

In [6]:
parsed = filtered.map(cast)

In [7]:
columns = parsed.map(lambda x: x[2]).distinct().collect()
basedict = {metric:0.0 for metric in columns}
ordered = OrderedDict(sorted(basedict.items(), key=lambda y: y[0]))

In [8]:
ordered

OrderedDict([('boottime', 0.0), ('bytes_in', 0.0), ('bytes_out', 0.0), ('cpu_aidle', 0.0), ('cpu_idle', 0.0), ('cpu_intr', 0.0), ('cpu_nice', 0.0), ('cpu_num', 0.0), ('cpu_sintr', 0.0), ('cpu_speed', 0.0), ('cpu_steal', 0.0), ('cpu_system', 0.0), ('cpu_user', 0.0), ('cpu_wio', 0.0), ('current_ps1', 0.0), ('current_ps2', 0.0), ('disk_free', 0.0), ('disk_free_absolute_dev_shm', 0.0), ('disk_free_absolute_rcvy', 0.0), ('disk_free_absolute_rootfs', 0.0), ('disk_free_absolute_scratch', 0.0), ('disk_free_absolute_var', 0.0), ('disk_free_absolute_var-rcvy', 0.0), ('disk_free_percent_dev_shm', 0.0), ('disk_free_percent_rcvy', 0.0), ('disk_free_percent_rootfs', 0.0), ('disk_free_percent_scratch', 0.0), ('disk_free_percent_var', 0.0), ('disk_free_percent_var-rcvy', 0.0), ('disk_total', 0.0), ('diskstat_sda_io_time', 0.0), ('diskstat_sda_percent_io_time', 0.0), ('diskstat_sda_read_bytes_per_sec', 0.0), ('diskstat_sda_read_time', 0.0), ('diskstat_sda_reads', 0.0), ('diskstat_sda_reads_merged', 0.0

In [9]:
def combine(u1, u2):
  u1.update(u2)
  return u1

def sequential(u, v):
  if not u:
      u = {}
  u[v[2]] = v[3]
  return u

def mergedicts(new, canon):
  tmp = canon.copy()
  tmp.update(new[1])
  return new[0], tmp

In [10]:
pivot = parsed.keyBy(lambda row: (row[0], row[1])).aggregateByKey(None, sequential, combine)

final = pivot.map(partial(mergedicts, canon=ordered))

In [19]:
final_ordered = final.coalesce(4).takeOrdered(100, key=lambda x: x[0])

In [20]:
final_ordered[9][0]

(1398382547, 'oss02')

In [ ]:
final_sorted = final.sortByKey(keyfunc= lambda k: k[0])

In [ ]:
final_dict = final_sorted.coalesce(4).collect()

In [14]:
final_ordered[-1][1]

OrderedDict([('boottime', 1397485868.0), ('bytes_in', 4078.61), ('bytes_out', 54009.39), ('cpu_aidle', 0.0), ('cpu_idle', 99.8), ('cpu_intr', 0.0), ('cpu_nice', 0.0), ('cpu_num', 24.0), ('cpu_sintr', 0.0), ('cpu_speed', 2793.0), ('cpu_steal', 0.0), ('cpu_system', 0.2), ('cpu_user', 0.0), ('cpu_wio', 0.0), ('current_ps1', 0.6), ('current_ps2', 0.48), ('disk_free', 150.084), ('disk_free_absolute_dev_shm', 23.478222), ('disk_free_absolute_rcvy', 14.780991), ('disk_free_absolute_rootfs', 13.091232), ('disk_free_absolute_scratch', 119.408478), ('disk_free_absolute_var', 6.921577), ('disk_free_absolute_var-rcvy', 7.331879), ('disk_free_percent_dev_shm', 99.908379), ('disk_free_percent_rcvy', 93.853882), ('disk_free_percent_rootfs', 83.124535), ('disk_free_percent_scratch', 94.774895), ('disk_free_percent_var', 87.899261), ('disk_free_percent_var-rcvy', 93.10981), ('disk_total', 161.176), ('diskstat_sda_io_time', 0.0), ('diskstat_sda_percent_io_time', 0.0), ('diskstat_sda_read_bytes_per_sec',